In [1]:
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import numpy as np

from tempfile import TemporaryFile
import os
import pickle
import random
import operator

import math

In [19]:
def distance (inst1, inst2, k) :
    dist = 0
    mm1 = inst1[0]
    cm1 = inst1[1]
    mm2 = inst2[0]
    cm2 = inst2[1]
    dist = np.trace(np.dot(np.linalg.inv(cm2), cm1))
    dist += (np.dot(np.dot((mm2-mm1).transpose(), np.linalg.inv(cm2)), mm2-mm1))
    dist += np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    dist -= k
    return dist

In [21]:
def get_neighbors (x_test, inst, k) :
    dist = []
    for x in range(len(x_test)) :
        val = distance(x_test[x], inst, k) + distance(inst, x_test[x], k)
        dist.append((x_test[x][2], val))
    dist.sort(key=lambda x: x[1])
    neighbors_list = []
    for x in range(k) :
        neighbors_list.append(dist[x][0])
    return neighbors_list

In [27]:
def nearest_neighbors(neighbors_list) :
    arr = {}
    
    for x in range(len(neighbors_list)) :
        val = neighbors_list[x]
        if val in arr :
            arr[val] += 1
        else :
            arr[val] = 1
        
    sorter = sorted(arr.items(), key=operator.itemgetter(1), reverse=True)
    return sorter[0][0]

In [4]:
def model_accuracy(y_test, y_predict) :
    correct = 0
    for x in range(len(y_test)) :
        if y_test[x][-1] == y_predict[x] :
            correct += 1
    
    return 1.0 * correct / len(y_test)

In [9]:
dataset_path = "./archive/Data/genres_original/"
dataset_file = open("extracted_data.dat", 'wb')
i = 0

for folder in os.listdir(dataset_path) :
    i += 1
    if i == 11 :
        break
    for file in os.listdir(dataset_path + folder) :
        try :
            (rate, sig) = wav.read(dataset_path + folder + "/" + file)
            mfcc_feat = mfcc(sig, rate, winlen = 0.020, appendEnergy=False)
            covariance = np.cov(np.matrix.transpose(mfcc_feat))
            mean_matrix = mfcc_feat.mean(0)
            feature = (mean_matrix, covariance, i)
            pickle.dump(feature, dataset_file)
        except Exception as e :
            print("exception: ",e,"in folder: ", folder, "in file: ", file)

dataset_file.close() 

exception:  File format b'\xcb\x15\x1e\x16' not understood. Only 'RIFF' and 'RIFX' supported. in folder:  jazz in file:  jazz.00054.wav


In [13]:
dataset = []
def load_dataset (filename, split, train_set, test_set) :
    with open("extracted_data.dat", 'rb') as file :
        while True :
            try :
                dataset.append(pickle.load(file))
            except EOFError :
                file.close()
                break
            
    for x in dataset :
        if random.random() < split :
            train_set.append(x)
        else :
            test_set.append(x)

train_set = []
test_set = []
load_dataset("extracted_data.dat", 0.66, train_set, test_set)

In [28]:
length = len(test_set)
predict = []
for x in range(len(test_set)) :
    predict.append(nearest_neighbors(get_neighbors(test_set, test_set[x], 5)))
    
accuracy = model_accuracy(test_set, predict)
print(accuracy)

0.8244514106583072
